In [ ]:
import scDML
print(scDML.__version__)
import scanpy as sc 
import numpy as np
from scDML import scDMLModel
from scDML.utils import print_dataset_information
import matplotlib.pyplot as plt
import anndata

import os
os.system("clear")

In [ ]:
for i in range(1, 31):
	adata = anndata.read_h5ad(f"../../../simulation/data/data{i}.h5ad")
	adata.X = adata.X - adata.X.min()
	model = scDMLModel(save_dir="./tmp/", verbose=False)
	sc.pp.normalize_total(adata,target_sum=1e4)
	sc.pp.log1p(adata)
	sc.pp.scale(adata)
	sc.tl.pca(adata)
	sc.pp.neighbors(adata)
	sc.tl.louvain(adata,resolution=1,key_added="init_cluster")
	model.integrate(adata,batch_key="Batch",ncluster_list=[4],
					expect_num_cluster=4,merge_rule="rule2")
	n_pc = 32
	res = adata.obsm["X_emb"] @ np.diag(adata.uns["pca"]["variance_ratio"][:n_pc]) @ adata.varm["PCs"][:,:n_pc].T
	res = anndata.AnnData(X=res, obs=adata.obs)
	res.write(f"../results/scdml_simu{i}.h5ad")